# 选择数据

In [1]:
import pandas as pd
import re

# 读取数据
df = pd.read_csv("data/raw/comments.csv")
df.head(1)

,_id,comment,hotel_id,hotel_name,images,publish_date,publish_location,review_count,room_type,score,stay_date,total,travel_type,useful_count,user_name
0,67fbcdb379a17df6ac05f9b1,酒店设施完善，性价比高，wifi给力，环境不错，地理位置好，附近逛街吃饭方便，房间很好非常喜...,1286148,北京索菲特大酒店,"[ ""https://dimg04.c-ctrip.com/images/0236e1200...",2025年3月15日,上海,3条点评,豪华大床房,5.0,于2025年3月入住,5,商务出差,1,_WeChat58639****


In [2]:
# 查看基础信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414330 entries, 0 to 414329
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   _id               414330 non-null  object 
 1   comment           414330 non-null  object 
 2   hotel_id          414330 non-null  int64  
 3   hotel_name        414330 non-null  object 
 4   images            414330 non-null  object 
 5   publish_date      414330 non-null  object 
 6   publish_location  414330 non-null  object 
 7   review_count      413784 non-null  object 
 8   room_type         414303 non-null  object 
 9   score             414330 non-null  float64
 10  stay_date         414330 non-null  object 
 11  total             414330 non-null  int64  
 12  travel_type       414330 non-null  object 
 13  useful_count      414330 non-null  int64  
 14  user_name         414330 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 47.4+ MB


In [3]:
# 从 stay_date 中提取年份
def extract_year(date_str):
    if pd.isna(date_str):
        return None
    match = re.search(r'(\d{4})', str(date_str))
    return int(match.group(1)) if match else None

df['year'] = df['stay_date'].apply(extract_year)

# 筛选 2023-2025 年的数据
filtered_df = df[df['year'].isin([2023, 2024, 2025])].copy()
filtered_df.drop('year', axis=1, inplace=True)

# 按 hotel_id 分组汇总
result = filtered_df.groupby('hotel_id').agg({
    'hotel_name': 'first',  # 取该 ID 对应的第一个酒店名
    'comment': 'count',     # 统计评论行数
    'score': 'mean'         # 计算平均分
})

# 重命名列名
result.columns = ['酒店名称', '评论总数', '平均得分']

# 按评论总数降序排列，取前 20 名
top_20 = result.sort_values(by='评论总数', ascending=False).head(20)

# 打印结果
print("2023-2025 年评论总数排名前 20 的酒店：")
print(top_20.to_string(formatters={'平均得分': '{:,.2f}'.format}))

2023-2025 年评论总数排名前 20 的酒店：
                           酒店名称  评论总数 平均得分
hotel_id                                  
1520200      加雅酒店（西安钟鼓楼五路口地铁站店）  6495 4.97
95472738         上海国际旅游度假区荷逸唐酒店  4254 4.97
95685502       深圳宝安机场RANZ兰兹国际酒店  4165 4.96
2073236    美丽豪酒店(西安小寨大雁塔大唐不夜城店)  4133 4.97
106876528            上海外滩滨江珍宝酒店  3481 4.93
6542110     桔子水晶上海国际旅游度假区周浦万达酒店  3443 4.97
446553                 广东地质山水酒店  3184 4.97
100589920            深圳福田皇岗城际酒店  3164 4.90
378770             上海佘山茂御臻品之选酒店  3147 4.92
468726        禧唐酒店(西安大雁塔大唐不夜城店)  3073 4.93
373052              上海浦东香格里拉大酒店  2994 4.83
79595121       华怡酒店（深圳宝安坪洲地铁站店）  2985 4.94
456133                   深圳维景酒店  2810 4.90
62631042         雅致酒店(珠江新城广州塔店)  2805 4.86
71632616       云和夜泊酒店(西安钟鼓楼南门店)  2719 4.97
344857                   广州花园酒店  2607 4.77
1286148                北京索菲特大酒店  2477 4.88
371239               金茂深圳JW万豪酒店  2380 4.93
21748036         西安大雁塔大唐不夜城雅致酒店  2266 4.94
102306108         广州天河太古汇喆啡锐品酒店  2180 4.85


In [4]:
# 筛选出 “广州花园酒店” 相关数据
target_hotel = "广州花园酒店"
target_col = ['_id', 'comment', 'images', 'score', 'publish_date', 'room_type', 'travel_type', 'useful_count', 'review_count']
garden_hotel_df = filtered_df[filtered_df['hotel_name'] == target_hotel][target_col]
garden_hotel_df.head(1)

,_id,comment,images,score,publish_date,room_type,travel_type,useful_count,review_count
347881,68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,"[ ""https://dimg04.c-ctrip.com/images/0230y1200...",5.0,2025年4月5日,红棉大床套房,家庭亲子,0,7条点评


In [5]:
# 保存数据
output_file = "data/raw/guangzhou_garden_hotel_comments_2023-2025.csv"
garden_hotel_df.to_csv(output_file, index=False, encoding='utf_8_sig')
print(f"{target_hotel}近三年的评论数据已保存: {output_file}")

广州花园酒店近三年的评论数据已保存: data/raw/guangzhou_garden_hotel_comments_2023-2025.csv
